##  原始数据格式数据使用numpy.loadtxt读取

In [175]:
from numpy import loadtxt
tmp = loadtxt('../data/fm_train_real.dat').T

## libsvm格式数据使用svmutil.svm_read_problem读取，这个最难用了

In [395]:
import sys
sys.path.append('/home/bd-dev/lijian/201801_ICML/script/libsvm/libsvm-3.22/python')
from svmutil import *
from svm import *
test = svm_read_problem('/home/bd-dev/lijian/201801_ICML/tmp/iris.scale')

##  libsvm格式数据使用sklearn带的方法读取，这个好用

In [394]:
from sklearn.externals.joblib import Memory
from sklearn.datasets import load_svmlight_file
mem = Memory("./mycache")

@mem.cache
def get_data():
    data = load_svmlight_file("/home/bd-dev/lijian/201801_ICML/tmp/iris.scale")
    return data[0], data[1]

X, y = get_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## libsvm格式数据使用shogun带的方法读取，这个一般 

In [389]:
from tempfile import NamedTemporaryFile
from shogun import SparseRealFeatures
from shogun import LibSVMFile
fname = '/home/bd-dev/lijian/201801_ICML/tmp/iris.scale'
f=SparseRealFeatures()
lab=f.load_with_labels(LibSVMFile(fname))
tmp_file = NamedTemporaryFile(suffix='svmlight')
f.save_with_labels(LibSVMFile(tmp_file.name, 'w'), lab)
f.d
print type(f)
print type(lab)

<type 'shogun.SparseRealFeatures'>
<type 'numpy.ndarray'>


## .mat数据格式使用scipy.io.load读取

In [9]:
import scipy.io as sio
data = sio.loadmat('/home/bd-dev/lijian/201801_ICML/tmp/RawData/plant/plant.phylpro.mat');
data['phylpros'].shape
X=data['phylpros']

data=sio.loadmat('/home/bd-dev/lijian/201801_ICML/data/plant/label_plant.mat');
y=data['y'];

## 分片->生成核矩阵
### 样本交叉验证时维度为(n,d)，生成核矩阵时维度为(d,n)。标签的数据格式是float64，且标签维度:(n,1)->(n,)

In [15]:
print X_train.T.shape
print y_train.shape

(89, 752)
(752, 1)


In [11]:
from shogun import RealFeatures, MulticlassLabels
from shogun import GaussianKernel
from shogun import LibSVM, KernelMulticlassMachine, MulticlassOneVsRestStrategy
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

width=2.1
C=1
epsilon=1e-5

feats_train=RealFeatures(X_train.T)
feats_test=RealFeatures(X_test.T)
kernel=GaussianKernel(feats_train, feats_train, width)

labels=MulticlassLabels(y_train.reshape(y_train.size,).astype('float64'))

classifier = LibSVM()
classifier.set_epsilon(epsilon)
#print labels.get_labels()
mc_classifier = KernelMulticlassMachine(MulticlassOneVsRestStrategy(),kernel,classifier,labels)
mc_classifier.train()

kernel.init(feats_train, feats_test)
out = mc_classifier.apply().get_labels()

In [5]:
#!/usr/bin/env python

import sys
sys.path.append('/home/bd-dev/lijian/201801_ICML/script/libsvm/libsvm-3.22/python')
from sklearn.externals.joblib import Memory
from sklearn.datasets import load_svmlight_file
mem = Memory("./mycache")

import scipy.io as sio
import numpy as np
from svmutil import *
from svm import *
from shogun import RealFeatures, MulticlassLabels
from shogun import GaussianKernel
from shogun import LibSVM, KernelMulticlassMachine, MulticlassOneVsRestStrategy
from sklearn.model_selection import train_test_split

# OneVsRest learning machine
def classifier_multiclassmachine (fm_train_real,fm_test_real,label_train_multiclass,width=2.1,C=1,epsilon=1e-5):
    feats_train=RealFeatures(fm_train_real)
    feats_test=RealFeatures(fm_test_real)
    kernel=GaussianKernel(feats_train, feats_train, width)

    labels=MulticlassLabels(label_train_multiclass)

    classifier = LibSVM()
    classifier.set_epsilon(epsilon)
    #print labels.get_labels()
    mc_classifier = KernelMulticlassMachine(C, MulticlassOneVsRestStrategy(),kernel,classifier,labels)
    mc_classifier.train()

    kernel.init(feats_train, feats_test)
    out = mc_classifier.apply().get_labels()

    return out

# load data from txt
def loadFromTxt (file_path):
    return np.loadtxt(file_path)

# load data from libsvm data
def loadFromLibsvm (file_path):
    sys.path.append('/home/bd-dev/lijian/201801_ICML/script/libsvm/libsvm-3.22/python')
    return svm_read_problem(file_path)

# load data from .mat file
def loadFromLibsvm (file_path):
    return sio.loadmat(file_path)

# phylpro type file        
def FinalResult4phylpro(data_name):
    data_path = '/home/bd-dev/lijian/201801_ICML/tmp/RawData/'+ data_name +'/'+ data_name +'.phylpro.mat'
    label_path = '/home/bd-dev/lijian/201801_ICML/data/' + data_name + '/label_' + data_name + '.mat'
    data = sio.loadmat(data_path)['phylpros']
    label = sio.loadmat(label_path)['y']
    label=label-label.min()

    accuracy=[]
    for i in range(times):
        X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=test_size)
        label_pre = classifier_multiclassmachine(X_train.T, X_test.T, y_train.reshape(y_train.size,).astype('float64'), width, C, epsilon)
        accuracy.append((y_test.reshape(y_test.size,).astype('float64')==label_pre).sum()/float(label_pre.size))
        print 'finish round ' + str(i) +', accuracy: ' + str(accuracy[len(accuracy)-1])
    print("\n".join(str(item) for item in accuracy))

@mem.cache
# svmlight type file        
def FinalResult4svmlight(data_name):
    print 'cj'
    data_path = '/home/bd-dev/lijian/201801_ICML/tmp/'+ data_name +'.scale'
    data = load_svmlight_file(data_path)
    X, y = data[0], data[1].reshape(len(data[1]),1)
    y=y-y.min()

    accuracy=[]
    for i in range(times):        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        label_pre = classifier_multiclassmachine(X_train.todense().T, X_test.todense().T, y_train.reshape(y_train.size,).astype('float64'), width, C, epsilon)
        accuracy.append((y_test.reshape(y_test.size,).astype('float64')==label_pre).sum()/float(label_pre.size))
        print 'finish round ' + str(i) +', accuracy: ' + str(accuracy[len(accuracy)-1])
    print("\n".join(str(item) for item in accuracy))

In [6]:
if __name__=='__main__':
    times=30
    C=100
    test_size=0.2
    width=8
    epsilon=1e-5
    FinalResult4svmlight('iris')    
    print 'test'

test


In [2]:
#!/usr/bin/env python
import sys
sys.path.append('/home/bd-dev/lijian/201801_ICML/script/libsvm/libsvm-3.22/python')
from sklearn.externals.joblib import Memory
from sklearn.datasets import load_svmlight_file
mem = Memory("./mycache")

import scipy.io as sio
import numpy as np
from svmutil import *
from svm import *
from shogun import LibSVM, KernelMulticlassMachine, MulticlassOneVsRestStrategy
from shogun import CombinedFeatures, RealFeatures, MulticlassLabels
from shogun import CombinedKernel, GaussianKernel, LinearKernel,PolyKernel
from shogun import MKLMulticlass
from shogun import GMNPSVM, CSVFile
from sklearn.model_selection import train_test_split
from shogun import MulticlassLibLinear
from sklearn.model_selection import KFold

# load data from txt
def loadFromTxt (file_path):
    return np.loadtxt(file_path)

# load data from libsvm data
def loadFromLibsvm (file_path):
    sys.path.append('/home/bd-dev/lijian/201801_ICML/script/libsvm/libsvm-3.22/python')
    return svm_read_problem(file_path)

# load data from .mat file
def loadFromLibsvm (file_path):
    return sio.loadmat(file_path)

# OneVsRest learning machine
def classifier_multiclassmachine (fm_train_real,fm_test_real,label_train_multiclass,width, C, epsilon):
    feats_train=RealFeatures(fm_train_real)
    feats_test=RealFeatures(fm_test_real)
    print '2 in'
    kernel=GaussianKernel(feats_train, feats_train, width)
    print '2 out'
    
    labels=MulticlassLabels(label_train_multiclass)

    classifier = LibSVM()
    classifier.set_epsilon(epsilon)
    mc_classifier = KernelMulticlassMachine(MulticlassOneVsRestStrategy(),kernel,classifier,labels)
    mc_classifier.train()

    kernel.init(feats_train, feats_test)
    out = mc_classifier.apply().get_labels()
    return out

# mc-mkl learning machine
def mkl_multiclass (fm_train_real, fm_test_real, label_train_multiclass,
    C, epsilon, num_threads, mkl_epsilon, mkl_norm):
    kernel = CombinedKernel()
    feats_train = CombinedFeatures()
    feats_test = CombinedFeatures()

    for i in range(-10,11):
        subkfeats_train = RealFeatures(fm_train_real)
        subkfeats_test = RealFeatures(fm_test_real)
        subkernel = GaussianKernel(pow(2,i+1))
        feats_train.append_feature_obj(subkfeats_train)
        feats_test.append_feature_obj(subkfeats_test)
        kernel.append_kernel(subkernel)

    kernel.init(feats_train, feats_train)

    labels = MulticlassLabels(label_train_multiclass)

    mkl = MKLMulticlass(C, kernel, labels)

    mkl.set_epsilon(epsilon)
    mkl.parallel.set_num_threads(num_threads)
    mkl.set_mkl_epsilon(mkl_epsilon)
    mkl.set_mkl_norm(mkl_norm)

    mkl.train()

    kernel.init(feats_train, feats_test)

    out =  mkl.apply().get_labels()
    return out

# multi-class classification based on C&S formulation
def classifier_multiclassliblinear (fm_train_real,fm_test_real,label_train_multiclass,width, C, epsilon):
    print C
    feats_train=RealFeatures(fm_train_real)
    feats_test=RealFeatures(fm_test_real)

    labels=MulticlassLabels(label_train_multiclass)

    classifier = MulticlassLibLinear(C,feats_train,labels)
    classifier.train()

    label_pred = classifier.apply(feats_test)
    out = label_pred.get_labels()
    return out

# multi-class on gmnp
def classifier_gmnpsvm (fm_train_real,fm_test_real,label_train_multiclass,width, C, epsilon):
    feats_train=RealFeatures(fm_train_real)
    feats_test=RealFeatures(fm_test_real)
    kernel=GaussianKernel(feats_train, feats_train, width)

    labels=MulticlassLabels(label_train_multiclass)

    svm=GMNPSVM(C, kernel, labels)
    svm.set_epsilon(epsilon)
    svm.train(feats_train)

    out=svm.apply(feats_test).get_labels()
    return out

def phylproFile(data_name, mode):
    data_path = '/home/bd-dev/lijian/201801_ICML/tmp/RawData/'+ data_name +'/'+ data_name +'.phylpro.mat'
    label_path = '/home/bd-dev/lijian/201801_ICML/data/' + data_name + '/label_' + data_name + '.mat'
    data = sio.loadmat(data_path)['phylpros']
    label = sio.loadmat(label_path)['y']
    label=label-label.min()

    accuracy=[]
    for i in range(times):
        X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=test_size)
        accuracy.append(train_test(mode, X_train.T, X_test.T, y_train.reshape(y_train.size,).astype('float64'), y_test.reshape(y_test.size,).astype('float64')))
        print 'finish '+ data_name + ' in ' + mode + ', round ' + str(i) +', accuracy: ' + str(accuracy[len(accuracy)-1])
    print 'mean accuracy of ' + data_name + ' in ' + mode + ' is ' + str(np.mean(accuracy))
    return accuracy

def svmlightFile(data_name, mode):
    data_path = '/home/bd-dev/lijian/201801_ICML/tmp/'+ data_name +'.scale'
    data = load_svmlight_file(data_path)
    X, y = data[0], data[1].reshape(len(data[1]),1)
    y=y-y.min()
    if data_name=='glass' or data_name=='svmguide4':    
        y = np.array([(x-1 if x>3 else x) for x in y])

    accuracy=[]
    for i in range(times):        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)        
        accuracy.append(train_test(mode, X_train.todense().T, X_test.todense().T, y_train.reshape(y_train.size,).astype('float64'), y_test.reshape(y_test.size,).astype('float64')))
        print 'finish '+ data_name + ' in ' + mode + ', round ' + str(i) +', accuracy: ' + str(accuracy[len(accuracy)-1])
    print 'mean accuracy of ' + data_name + ' in ' + mode + ' is ' + str(np.mean(accuracy))
    return accuracy

def train_test(mode, X_train, X_test, y_train, y_test):
    if mode=='mcmkl':
        label_pre = mkl_multiclass(X_train, X_test, y_train, C, epsilon, num_threads, mkl_epsilon, mkl_norm)
    elif mode=='1vR':
        label_pre = classifier_multiclassmachine(X_train, X_test, y_train, width, C, epsilon)
    elif mode=='gmnp':
        label_pre = classifier_gmnpsvm(X_train, X_test, y_train, width, C, epsilon)
    elif mode=='cs':
        label_pre = classifier_multiclassliblinear(X_train, X_test, y_train, width, C, epsilon)    
    return  (y_test==label_pre).sum()/float(label_pre.size)

def cv_para_svmlight(data_name, mode, folds, C):
    data_path = '/home/bd-dev/lijian/201801_ICML/tmp/'+ data_name +'.scale'
    data = load_svmlight_file(data_path)
    X, y = data[0], data[1].reshape(len(data[1]),1)
    y=y-y.min()
    if data_name=='glass' or data_name=='svmguide4':    
        y = np.array([(x-1 if x>3 else x) for x in y])

    accuracy=[]
    for train_index, test_index in KFold(n_splits=folds).split(X):        
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]        
        accuracy.append(train_test(mode, X_train.todense().T, X_test.todense().T, y_train.reshape(y_train.size,).astype('float64'), y_test.reshape(y_test.size,).astype('float64')))
        print 'finish '+ data_name + ' in ' + mode + ', round ' + str(i) +', accuracy: ' + str(accuracy[len(accuracy)-1])
    print 'mean accuracy of ' + data_name + ' in ' + mode + ' is ' + str(np.mean(accuracy))
    return np.mean(accuracy)

def cv_para_phylpro(data_name, mode, folds, C):
    data_path = '/home/bd-dev/lijian/201801_ICML/tmp/RawData/'+ data_name +'/'+ data_name +'.phylpro.mat'
    label_path = '/home/bd-dev/lijian/201801_ICML/data/' + data_name + '/label_' + data_name + '.mat'
    data = sio.loadmat(data_path)['phylpros']
    label = sio.loadmat(label_path)['y']
    label=label-label.min()

    accuracy=[]
    for train_index, test_index in KFold(n_splits=folds).split(X):        
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]        
        accuracy.append(train_test(mode, X_train.T, X_test.T, y_train.reshape(y_train.size,).astype('float64'), y_test.reshape(y_test.size,).astype('float64')))
        print 'finish '+ data_name + ' in ' + mode + ', round ' + str(i) +', accuracy: ' + str(accuracy[len(accuracy)-1])
    print 'mean accuracy of ' + data_name + ' in ' + mode + ' is ' + str(np.mean(accuracy))
    return np.mean(accuracy)

def get_best_para(file_type, data_name, mode, para_list, folds):
    best_para = 0
    for para in para_list:
        C = para_list
        if file_type =='4':
            accuracy = cv_para_phylpro(data_name, mode, folds, C)
        elif file_type =='5':
            accuracy = cv_para_svmlight()
        if max_acc < accuracy:
            max_acc = accuracy
            best_para = C
    return best_para

In [ ]:
if __name__=='__main__':
    times=30
    C=100 #[1, 10, 100, 1000]    
    epsilon= 1e-3 #1e-5
    mkl_epsilon=0.001
    test_size=0.2
    width=8
    num_threads=8
    mkl_norm=2
    
    sys.argv=['ttt', '5', 'sector', 'gmnp', '10']
    if len(sys.argv) > 4:
        C = int(sys.argv[4])
    if len(sys.argv) > 5:
        times=int(sys.argv[5])
        bestC = 1
        maxMean = 0
        for i in [1, 10, 100, 1000]:
            C=i
            if sys.argv[1]=='4':
                accuracy = phylproFile(sys.argv[2],sys.argv[3])
            elif sys.argv[1]=='5':
                accuracy = svmlightFile(sys.argv[2],sys.argv[3])   
            if np.mean(accuracy) > maxMean:
                bestC = C
                maxMean = np.mean(accuracy)
        print 'Best para C is ' + str(bestC)
        C = bestC
        times = 30
                
    if sys.argv[1]=='4':
        accuracy = phylproFile(sys.argv[2],sys.argv[3])
    elif sys.argv[1]=='5':
        accuracy = svmlightFile(sys.argv[2],sys.argv[3])    
    print("\n".join(str(item*100) for item in accuracy))
    

In [5]:
#!/usr/bin/env python
import sys
sys.path.append('/home/bd-dev/lijian/201801_ICML/script/libsvm/libsvm-3.22/python')
from sklearn.externals.joblib import Memory
from sklearn.datasets import load_svmlight_file
mem = Memory("./mycache")

import scipy.io as sio
import numpy as np
from svmutil import *
from svm import *
from shogun import LibSVM, KernelMulticlassMachine, MulticlassOneVsRestStrategy
from shogun import CombinedFeatures, RealFeatures, MulticlassLabels
from shogun import CombinedKernel, GaussianKernel, LinearKernel,PolyKernel
from shogun import MKLMulticlass
from shogun import GMNPSVM, CSVFile
from sklearn.model_selection import train_test_split
from shogun import MulticlassLibLinear
from sklearn.model_selection import KFold

times=30
C=100 #[1, 10, 100, 1000]    
epsilon=1e-5
mkl_epsilon=0.001
test_size=0.2
width=8
num_threads=8
mkl_norm=2

# load data from txt
def loadFromTxt (file_path):
    return np.loadtxt(file_path)

# load data from libsvm data
def loadFromLibsvm (file_path):
    sys.path.append('/home/bd-dev/lijian/201801_ICML/script/libsvm/libsvm-3.22/python')
    return svm_read_problem(file_path)

# load data from .mat file
def loadFromLibsvm (file_path):
    return sio.loadmat(file_path)

# OneVsRest learning machine
def classifier_multiclassmachine (fm_train_real,fm_test_real,label_train_multiclass,width, C, epsilon):
    feats_train=RealFeatures(fm_train_real)
    feats_test=RealFeatures(fm_test_real)
    print '2 in'
    kernel=GaussianKernel(feats_train, feats_train, width)
    print '2 out'
    
    labels=MulticlassLabels(label_train_multiclass)

    classifier = LibSVM()
    classifier.set_epsilon(epsilon)
    mc_classifier = KernelMulticlassMachine(MulticlassOneVsRestStrategy(),kernel,classifier,labels)
    mc_classifier.train()

    kernel.init(feats_train, feats_test)
    out = mc_classifier.apply().get_labels()
    return out

# mc-mkl learning machine
def mkl_multiclass (fm_train_real, fm_test_real, label_train_multiclass,
    C, epsilon, num_threads, mkl_epsilon, mkl_norm):
    kernel = CombinedKernel()
    feats_train = CombinedFeatures()
    feats_test = CombinedFeatures()

    for i in range(-10,11):
        subkfeats_train = RealFeatures(fm_train_real)
        subkfeats_test = RealFeatures(fm_test_real)
        subkernel = GaussianKernel(pow(2,i+1))
        feats_train.append_feature_obj(subkfeats_train)
        feats_test.append_feature_obj(subkfeats_test)
        kernel.append_kernel(subkernel)

    kernel.init(feats_train, feats_train)

    labels = MulticlassLabels(label_train_multiclass)

    mkl = MKLMulticlass(C, kernel, labels)

    mkl.set_epsilon(epsilon)
    mkl.parallel.set_num_threads(num_threads)
    mkl.set_mkl_epsilon(mkl_epsilon)
    mkl.set_mkl_norm(mkl_norm)

    mkl.train()

    kernel.init(feats_train, feats_test)

    out =  mkl.apply().get_labels()
    return out

# multi-class classification based on C&S formulation
def classifier_multiclassliblinear (fm_train_real,fm_test_real,label_train_multiclass,width, C, epsilon):
	feats_train=RealFeatures(fm_train_real)
	feats_test=RealFeatures(fm_test_real)

	labels=MulticlassLabels(label_train_multiclass)

	classifier = MulticlassLibLinear(C,feats_train,labels)
	classifier.train()

	label_pred = classifier.apply(feats_test)
	out = label_pred.get_labels()
	return out

# multi-class on gmnp
def classifier_gmnpsvm (fm_train_real,fm_test_real,label_train_multiclass,width, C, epsilon):
    feats_train=RealFeatures(fm_train_real)
    feats_test=RealFeatures(fm_test_real)
    kernel= GaussianKernel(feats_train, feats_train, width)

    labels=MulticlassLabels(label_train_multiclass)

    print type(C), type(kernel), type(labels)
    svm=GMNPSVM(C, kernel, labels)
    svm.set_epsilon(epsilon)
    print 'start train'
    svm.train(feats_train)
    print 'finish train'

    out=svm.apply(feats_test).get_labels()
    return out

def phylproFile(data_name, mode):
    data_path = '/home/bd-dev/lijian/201801_ICML/tmp/RawData/'+ data_name +'/'+ data_name +'.phylpro.mat'
    label_path = '/home/bd-dev/lijian/201801_ICML/data/' + data_name + '/label_' + data_name + '.mat'
    data = sio.loadmat(data_path)['phylpros']
    label = sio.loadmat(label_path)['y']
    label=label-label.min()

    accuracy=[]
    for i in range(times):
        X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=test_size)
        accuracy.append(train_test(mode, X_train.T, X_test.T, y_train.reshape(y_train.size,).astype('float64'), y_test.reshape(y_test.size,).astype('float64')))
        print 'finish '+ data_name + ' in ' + mode + ', round ' + str(i) +', accuracy: ' + str(accuracy[len(accuracy)-1])
    print 'mean accuracy of ' + data_name + ' in ' + mode + ' is ' + str(np.mean(accuracy))
    return accuracy

def svmlightFile(data_name, mode):
    data_path = '/home/bd-dev/lijian/201801_ICML/tmp/'+ data_name +'.scale'
    data = load_svmlight_file(data_path)
    X, y = data[0], data[1].reshape(len(data[1]),1)
    y=y-y.min()
    if data_name=='glass' or data_name=='svmguide4':    
        y = np.array([(x-1 if x>3 else x) for x in y])

    accuracy=[]
    for i in range(times):        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)        
        accuracy.append(train_test(mode, X_train.todense().T, X_test.todense().T, y_train.reshape(y_train.size,).astype('float64'), y_test.reshape(y_test.size,).astype('float64')))
        print 'finish '+ data_name + ' in ' + mode + ', round ' + str(i) +', accuracy: ' + str(accuracy[len(accuracy)-1])
    print 'mean accuracy of ' + data_name + ' in ' + mode + ' is ' + str(np.mean(accuracy))
    return accuracy

def train_test(mode, X_train, X_test, y_train, y_test, C=C):
    print 'train_test C:' + str(C)
    if mode=='mcmkl':
        label_pre = mkl_multiclass(X_train, X_test, y_train, C, epsilon, num_threads, mkl_epsilon, mkl_norm)
    elif mode=='1vR':
        label_pre = classifier_multiclassmachine(X_train, X_test, y_train, width, C, epsilon)
    elif mode=='gmnp':
        label_pre = classifier_gmnpsvm(X_train, X_test, y_train, width, C, epsilon)
    elif mode=='cs':
        label_pre = classifier_multiclassliblinear(X_train, X_test, y_train, width, C, epsilon)    
    return  (y_test==label_pre).sum()/float(label_pre.size)

def cv_para_svmlight(C):
    data_path = '/home/bd-dev/lijian/201801_ICML/tmp/'+ data_name +'.scale'
    data = load_svmlight_file(data_path)
    X, y = data[0], data[1].reshape(len(data[1]),1)
    y=y-y.min()
    if data_name=='glass' or data_name=='svmguide4':    
        y = np.array([(x-1 if x>3 else x) for x in y])
    if len(y) > 3000:
        X = X[:3000]
        y = y[:3000]

    accuracy=[]
    for train_index, test_index in KFold(n_splits=folds, shuffle=True).split(y):        
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
        print 'svmlight C:' + str(C)
        accuracy.append(train_test(mode, X_train.todense().T, X_test.todense().T, y_train.reshape(y_train.size,).astype('float64'), y_test.reshape(y_test.size,).astype('float64'),C))
    print 'mean accuracy of ' + data_name + ' in ' + mode + ' is ' + str(np.mean(accuracy))
    return np.mean(accuracy)

def cv_para_phylpro(C):
    data_path = '/home/bd-dev/lijian/201801_ICML/tmp/RawData/'+ data_name +'/'+ data_name +'.phylpro.mat'
    label_path = '/home/bd-dev/lijian/201801_ICML/data/' + data_name + '/label_' + data_name + '.mat'
    data = sio.loadmat(data_path)['phylpros']
    label = sio.loadmat(label_path)['y']
    label=label-label.min()

    accuracy=[]
    for train_index, test_index in KFold(n_splits=folds, shuffle=True).split(X):        
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]        
        accuracy.append(train_test(mode, X_train.T, X_test.T, y_train.reshape(y_train.size,).astype('float64'), y_test.reshape(y_test.size,).astype('float64'),C))
    print 'mean accuracy of ' + data_name + ' in ' + mode + ' is ' + str(np.mean(accuracy))
    return np.mean(accuracy)

def get_best_para():
    best_para = 0
    max_acc = 0
    for para in para_list:
        C = para
        if file_type =='4':
            accuracy = cv_para_phylpro(C)
        elif file_type =='5':
            accuracy = cv_para_svmlight(C)
        if max_acc < accuracy:
            max_acc = accuracy
            best_para = C
    return best_para


In [23]:
data_path = '/home/bd-dev/lijian/201801_ICML/tmp/sector.scale'
data = load_svmlight_file(data_path)
X, y = data[0], data[1].reshape(len(data[1]),1)
y=y-y.min()

In [8]:
X.shape

(3000, 55197)

In [9]:
y.shape

(3000, 1)

In [10]:
np.unique(y)

array([   0.,    1.,    2.,    3.,    4.,    5.,    6.,    7.,    8.,
          9.,   10.,   11.,   12.,   13.,   14.,   15.,   16.,   17.,
         18.,   19.,   20.,   21.,   22.,   23.,   24.,   25.,   26.,
         27.,   28.,   29.,   30.,   31.,   32.,   33.,   34.,   35.,
         36.,   37.,   38.,   39.,   40.,   41.,   42.,   43.,   44.,
         45.,   46.,   47.,   48.,   49.,   50.,   51.,   52.,   53.,
         54.,   55.,   56.,   57.,   58.,   59.,   60.,   61.,   62.,
         63.,   64.,   65.,   66.,   67.,   68.,   69.,   70.,   71.,
         72.,   73.,   74.,   75.,   76.,   77.,   78.,   79.,   80.,
         81.,   82.,   83.,   84.,   85.,   86.,   87.,   88.,   89.,
         90.,   91.,   92.,   93.,   94.,   95.,   96.,   97.,   98.,
        100.,  101.,  102.,  103.,  104.])

In [12]:
len(np.unique(y))

104

In [27]:
kf = KFold(n_splits=int(1/(1-3000.0/len(y)))+1, shuffle=True).split(y)

In [34]:
a,b=kf

In [43]:
len(np.unique(y[a[1]]))

105

In [245]:
data_path = '/home/bd-dev/lijian/201801_ICML/tmp/sector.scale'
data = load_svmlight_file(data_path)
X, y = data[0], data[1].reshape(len(data[1]),1)
y=y-y.min()
X=X[(y<30).reshape(6412,),:]
y=y[(y<30).reshape(6412,),:]
print type(X)

<class 'scipy.sparse.csr.csr_matrix'>


In [231]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
new_data=pca.fit_transform(X.toarray())

In [226]:
print(pca.explained_variance_ratio_)  
print(type(pca.singular_values_))

[ 0.02343408  0.01119118  0.01087832  0.00980029  0.00820345  0.00657229
  0.00600863  0.00507656  0.00480313  0.00433513  0.00428639  0.00412836
  0.00405653  0.00386227  0.00363855  0.0035958   0.00344988  0.00344606
  0.00323247  0.00316549]
<type 'numpy.ndarray'>


In [247]:
import scipy
type(X)==scipy.sparse.csr.csr_matrix

True

In [254]:
para_list = np.logspace(-2, 12, 11, base=2)np.logspace(0,9,10,base=2)
print para_list

array([   1.,    2.,    4.,    8.,   16.,   32.,   64.,  128.,  256.,  512.])

In [262]:
np.logspace(-2,12,15,base=2)

array([  2.50000000e-01,   5.00000000e-01,   1.00000000e+00,
         2.00000000e+00,   4.00000000e+00,   8.00000000e+00,
         1.60000000e+01,   3.20000000e+01,   6.40000000e+01,
         1.28000000e+02,   2.56000000e+02,   5.12000000e+02,
         1.02400000e+03,   2.04800000e+03,   4.09600000e+03])